In [1]:
!pip install pinecone

In [2]:
!pip install cohere

In [3]:
from pinecone import Pinecone, ServerlessSpec
# import openai
import cohere
import os

In [4]:
#Index Creation
pc = Pinecone(
    api_key="pcsk_44R4w_9mXN3CD1j1kqzSjUzzwYkDJePke4GioyChxpYWmK48Ro369U4vGmEKd5rtpFkLi"
)
index_name = "cv-chunks"
embedding_dim = 4096

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",  # or "cosine", depending on your similarity metric preference
        spec=ServerlessSpec(
            cloud="aws",       # Change this to your cloud provider if necessary
            region="us-east-1" # Change to your region
        )
    )

# Connect to the index
index = pc.Index(index_name)

# Initialize OpenAI for embeddings
co = cohere.Client("m5XzuXbgRfMj9OqycoyofSbrOr2USUewghDXMLaa")

In [5]:
import time
def get_embedding(text: str, model="text-embedding-ada-002") -> list:
    """
    Function to embed CVs chunks
    :param text: CVs chunks
    :param model: Embedding model
    :return: Chunk Embeddings
    """
    while True:
        try:
            response = co.embed(texts=[text], model=model)
            return response.embeddings[0]
        except cohere.TooManyRequestsError:
            print("⚠️ Rate limit reached! Waiting before retrying...")
            time.sleep(60)  # Wait 60 seconds before retrying

In [6]:
import json
def uploading_vector_db(file_path,model):
  """
  Function to upload embed the CVs chunks and upload it to a vector db(Pinecone) (RUN TO UPLOAD DATA ONLY)
  :param file_path: File containing CVs chunks
  :param model: Embedding model
  :return: None
  """
  with open(file_path, "r", encoding="utf-8") as f:
      cv_chunks = json.load(f)

  vectors = []
  for i, chunk in enumerate(cv_chunks):
      embedding = get_embedding(chunk["content"],model=model)
      index.upsert(vectors=[(
            chunk["chunk_id"],
            embedding,
            {
                "original_file": chunk["original_file"],
                "content": chunk["content"]
            }
        )])
      if (i + 1) % 90 == 0:
        print(f"⏸️ upload {i+1} embeddings. Pausing for 60 seconds...")
        time.sleep(60)


  print("Upserted CV chunk embeddings with metadata to Pinecone.")

In [7]:
CV_file_path = "/content/all_chunks.json"
uploading_vector_db(file_path=CV_file_path,model="embed-english-v2.0")

⏸️ upload 90 embeddings. Pausing for 60 seconds...
⏸️ upload 180 embeddings. Pausing for 60 seconds...
⏸️ upload 270 embeddings. Pausing for 60 seconds...
⏸️ upload 360 embeddings. Pausing for 60 seconds...
Upserted CV chunk embeddings with metadata to Pinecone.
